[個股前20大分點進出] 
網址 : https://api.tej.com.tw/columndoc.html?subId=150

In [1]:
import tejapi
from datetime import date
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import PatternFill

## API 拿

In [2]:
# 今天的日期
today = date.today().strftime("%Y-%m-%d")

tejapi.ApiConfig.api_key = "eJ8bhkYk3BEbRVtqNxRc0R45ZzCYaM"
tejapi.ApiConfig.api_base = "http://10.10.10.66"

# 需要的欄位
columns = ['coid', 'mdate', 'key3', 'zvol_d', 'zamt_d', 'net_s1', 'net_s1b', 'net_a1', 'net_a1b']
# columns = ['key3', 'zvol_d', 'zamt_d', 'net_s1', 'net_s1b']
# data = tejapi.get('TWN/ABSR20', mdate='2024-01-09', opts={'columns': columns}, chinese_column_name=True, paginate=True)
data = tejapi.get('TWN/ABSR20', mdate=today, opts={'columns': columns}, chinese_column_name=True, paginate=True)
data

,證券名稱,年月日,排名,成交量(股),成交值(元),當日買賣超量(股),當日買賣超量-券商,當日買賣超金額(元),當日買賣超金額-券商
None,,,,,,,,,
0,0050,2024-01-10 00:00:00+00:00,1,7249096.0,959855308.0,376689.0,永豐金,4.989792e+07,永豐金
1,0050,2024-01-10 00:00:00+00:00,2,7249096.0,959855308.0,224805.0,新光,2.974355e+07,新光
2,0050,2024-01-10 00:00:00+00:00,3,7249096.0,959855308.0,177196.0,凱基台北,2.348396e+07,凱基台北
3,0050,2024-01-10 00:00:00+00:00,4,7249096.0,959855308.0,156652.0,國泰綜合,2.075289e+07,國泰綜合
4,0050,2024-01-10 00:00:00+00:00,5,7249096.0,959855308.0,132927.0,凱基站前,1.760360e+07,凱基站前
...,...,...,...,...,...,...,...,...,...
48835,Y9999,2024-01-10 00:00:00+00:00,16,5764346.0,258068344.0,3126884.0,富邦台北,1.365793e+08,國泰敦南
48836,Y9999,2024-01-10 00:00:00+00:00,17,5764346.0,258068344.0,3113000.0,港商法興,1.361905e+08,元富台中
48837,Y9999,2024-01-10 00:00:00+00:00,18,5764346.0,258068344.0,3020264.0,玉山,1.291208e+08,凱基員林


## 排除分點清單

In [3]:
# 排除 (外資) (總公司)
lst_bname_foreign = ['摩根大通', '上海匯豐' ,'美林', '台灣摩根', '台灣匯立', '法銀巴黎', '花旗環球', '美林', '美商高盛', '港商野村', '港麥格理', '瑞銀', '聯邦']
lst_bname_corporate = ['元大', '兆豐', '元富', '台新', '永豐金', '玉山', '合庫', '亞東', '康和', '統一', '凱基', '凱基台北', '富邦', '富邦經紀', '群益','華南永昌'
                      ,'中國信託', '臺銀', '宏遠', '彰銀', '第一金證', '新光', '土銀', '國泰綜合']
lst_exclude = lst_bname_foreign + lst_bname_corporate

# 整理

In [6]:
def arrange_data(df):
    df['年月日'] = df['年月日'].dt.strftime('%Y-%m-%d')
    df = df[df['證券名稱'].apply(lambda x: len(str(x)) == 4)]
    df.columns = ['scode', 'date', '排名', 'vol(千股)', 'vol(萬)', 'buy_量', '券商分點_量', 'buy_金額', '券商分點_金額']

    # 把 公司中文名 merge
    df_name = pd.read_excel('comp_name.xlsx')
    df_name.rename(columns={'代號': 'scode', '名稱':'sname'}, inplace=True)
    df_name['scode'] = df_name['scode'].astype(str)
    df = df.merge(df_name, on=['scode'], how='left')
    df = df[df['sname'].notnull()]
    df = df[['scode', 'sname', 'date', '排名', 'vol(千股)', 'vol(萬)', 'buy_量', '券商分點_量', 'buy_金額', '券商分點_金額']]

    # 整理買賣超金額 / 股數 / 買超佔比
    df['buy_proportion'] = round(df['buy_量'] / df['vol(千股)']*100,2)
    df['vol(千股)'] = round(df['vol(千股)']/1000, 2)
    df['vol(萬)'] = round(df['vol(萬)']/10000, 2)
    df['buy_量'] = round(df['buy_量']/1000, 2)
    df['buy_金額'] = round(df['buy_金額']/10000, 2)

    # 整理 成交量
    df_temp = df[['scode', 'date', 'buy_量', '券商分點_量']].copy()
    df = df[['scode', 'sname', 'date', '排名', 'vol(千股)', 'vol(萬)', 'buy_金額', '券商分點_金額', 'buy_proportion']]
    df = df.merge(df_temp, left_on=['scode', 'date', '券商分點_金額'], right_on=['scode', 'date', '券商分點_量'], how='left')
    df = df[['scode', 'sname', 'date', '排名', 'vol(千股)', 'vol(萬)', 'buy_量', 'buy_金額', '券商分點_金額', 'buy_proportion']]
    df.rename(columns={'券商分點_金額': 'bname'}, inplace=True)

    # 篩選
    df = df[df['buy_金額']>100]

    df.reset_index(drop=True, inplace=True)
    return df

df = data.copy()
df = arrange_data(df)

# 條件 
# 1 : 買超金額 > ?
# 2 : 買超佔比 > ?
# 3 : 排除分點
df_out = df[(df['buy_金額']>300)&(~df['bname'].isin(lst_exclude))].copy()
df_out = df_out.sort_values(['buy_金額'],ascending=False).reset_index(drop=True)

# to excel
file_name = '分點買賣超_%s.xlsx'%(date.today().strftime("%Y%m%d"))
df_out.to_excel(file_name, index=False, header=True)

# 把 excel 著色
def color_excel(file_name):
    workbook = openpyxl.load_workbook(file_name)
    sheet = workbook.worksheets[0]

    # 顯示 cell 資料
    for i in range(1, sheet.max_row+1):
        for j in range(1, sheet.max_column+1):
            cell = sheet.cell(row=i, column=j)
            if i == 1:
                cell.font = Font(name='微軟正黑體', bold=True, size=12)
            elif j in [1,3,4,5,6,7]:
                cell.font = Font(name='Calibri')
            elif j in [2,9]:
                cell.font = Font(name='微軟正黑體')


            if j == 8 and i != 1: # buy_金額
                # print(i, j, cell.value)
                if cell.value > 20000:
                    color_fill = PatternFill(start_color='ffd6cf', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill          
                elif cell.value > 10000:
                    color_fill = PatternFill(start_color='ffe6b3', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                elif cell.value > 5000:
                    color_fill = PatternFill(start_color='c7dbf7', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                elif cell.value > 1000:
                    color_fill = PatternFill(start_color='DDF3FF', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                cell.font = Font(name='Calibri')
            elif j == 10 and i != 1: # buy_proportion
                if cell.value > 50:
                    color_fill = PatternFill(start_color='bfa87c', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                elif cell.value > 20:
                    color_fill = PatternFill(start_color='e6d395', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                elif cell.value > 10:
                    color_fill = PatternFill(start_color='f0e6c0', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                elif cell.value > 5:
                    color_fill = PatternFill(start_color='fffad9', fill_type='solid')
                    sheet.cell(row=i, column=j).fill = color_fill
                cell.font = Font(name='Calibri')

    workbook.save(file_name)
    
color_excel(file_name)

# 抓 google sheet (籌碼分點紀錄)

In [24]:
import pygsheets
import pandas as pd

gc = pygsheets.authorize(service_file='sotck-chip-analysis-4d71166d853b.json')
sheet = gc.open('chip')
print(sheet.worksheets()) # 看有哪些 sheet
worksheets = sheet[0]
df = pd.DataFrame(worksheets.get_all_records())
df

[<Worksheet '操作厲害' index:0>, <Worksheet '董監分點' index:1>, <Worksheet '誰' index:2>, <Worksheet '隔日沖' index:3>]


,scode,sname,分點,特性,Good,權證,年度1,年度2,備註,短線 : 1個月內\n波段 : 1~12個月
0,1216,統一,凱基中港,長線囤貨,,,2023-08,,云式公司派,
1,1310,台苯,元大忠孝鼎富,地緣/囤貨,V,,2023-04,2023-07,有點像公司派，不太確定,
2,1477,聚陽,華南永昌中正,公司派,,,2023-10,,云式公司派 / 庫藏股,
3,1504,士電,第一金高雄,公司派,,,2023-09,,云式公司派 / 庫藏股,
4,1504,東元,元大館前,波段囤貨,V,,2023-02,2023-06,云式公司派,
...,...,...,...,...,...,...,...,...,...,...
268,8421,旭源,兆豐新營,長波段囤貨,V,,,,,
269,1532,勤美,玉山台中,地緣,,,,,,
270,1532,勤美,台新台中,地緣,,,,,,
271,6715,嘉基,凱基站前,波段,,,,,,
